## Importation des librairies

In [1]:
#Libraries
import re
import pandas as pd

## Class Tache :

In [2]:
class Tache(object):
    def __init__(self, nom, description, precedents, duree):
        self.nom = nom
        self.description = description
        self.precedents = precedents
        self.duree = duree
        self.ES = 0
        self.EF = 0
        self.successeurs = []
        self.LS = 0
        self.LF = 0
        self.marge_totale = 0
        self.critique = ''
    #fonction pour la calcule de la marge totale et est ce que une tache est critique ou non
    def calcMargeTotale(self):
        self.marge_totale = self.LF - self.duree - self.ES
        if self.marge_totale > 0:
            self.critique = 'Non'
        else:
            self.critique = 'Oui'

## Declaration des fonctions :

lireData :

In [3]:
#fonction pour passer une table de donnée existe dans un fichier excel
#la table de donnée sera transformée en un dataframe a l'aide du librairie Pandas
#cette fonction retourne un dataframe
def lireData(excel_file):
    df = pd.read_excel(excel_file)
    return df

creerTaches :

In [4]:
#fonction permet de créer une liste des taches a partir du dataframe géneré par la fonction lireData
def creerTaches(df):
    liste_taches = []
    
    for i in range(len(df)):
        liste_taches.append(Tache(df['Tache'][i],df['Description'][i],df['P'][i],df['Duree'][i]))
    
    return (liste_taches)

parcoursEnAvant :

In [5]:
#fonction fait un parcours en avant est permet de calculer : ES (early start) et EF (early finish)
#il est obligatoire de passer une liste des taches

#règle 1 : ES des taches qui n'ont aucun precedents égale 0
#règle 2 : ES des taches qui one des taches qui ont des precedents égale au maximum des EF de ses precedents
#règle 3 : ES de la dernière tache égale à la somme des ES de toutes les activités plus la durée de la dernière

def parcoursEnAvant(liste_taches):
    for tache in liste_taches:
        if type(tache.precedents) is str:
            tache.precedents = tache.precedents.upper()
            ef = []
            for j in tache.precedents:
                for t in liste_taches:
                    if t.nom == j:
                        ef.append(t.EF)
                tache.ES = max(ef)
            del ef
        else:
            tache.ES = 0
        
        tache.EF = tache.ES + tache.duree

parcoursEnArriere :

In [6]:
#fonction fait un parcours en arrière est permet de calculer : LS (late start) et LF (late finish)
#il est obligatoire d'implementer la fonction parcoursEnAvant avant d'utiliser parcoursEnArrier
#règle 1 : LF d'une tache qui n'est pas precedente d'aucune tache égale le maximum des EF
#règle 2 : LF d'une tache qui precedente d'autres taches égale le minimum EF de ses successeurs
#règle 3 : LS d'une tache = lf - durée
def parcoursEnArriere(liste_taches):
    
    pred = []
    eF = []
    
    #obtenir les EF de toutes les taches et remplire pred
    #obtenir les successeurs de chaque tache
    for tache in liste_taches:
        if type(tache.precedents) == str:
            for j in tache.precedents:
                pattern = re.compile(r'[A-Z]')
                match = pattern.finditer(j)
                for r in match:
                    pred.append(j)
                    for m in liste_taches:
                        if m.nom == j:
                            m.successeurs.append(tache.nom)
        eF.append(tache.EF)
        
    for tache in reversed(liste_taches):
        if tache.nom not in pred:
            tache.LF = max(eF)
        else:
            
            minLs = []
            for x in tache.successeurs:
                for t in (liste_taches):
                    if t.nom == x:
                        minLs.append(t.LS)
            tache.LF = min(minLs)
            del minLs
            
        tache.LS = tache.LF - tache.duree

marge totale :

In [7]:
def margeTotale(liste_taches):
    for tache in liste_taches:
        tache.calcMargeTotale()

mise a jour du dataframe:

In [17]:
#update dataframe
def MAJdataFrame(df,liste_taches):
    df2 = pd.DataFrame({
        'Tache' : df['Tache'],
        'Description' : df['Description'],
        'P' : df['P'],
        'Duree' : df['Duree'],
        'Marge Totale' : pd.Series([tache.marge_totale for tache in liste_taches]),
        'Critique' : pd.Series([tache.critique for tache in liste_taches]),
    })
    return (df2)

fonction main :

In [19]:
def main():
    excel_file = 'input.xlsx'
    df = lireData(excel_file)
    print('loaded data:')
    print(df)
    liste_taches = creerTaches(df)
    parcoursEnAvant(liste_taches)
    parcoursEnArriere(liste_taches)
    margeTotale(liste_taches)
    finaldf = MAJdataFrame(df,liste_taches)
    print("results:")
    print(finaldf)
    print('results saved to pertcpm.xlsx')
    finaldf.to_excel('output.xlsx', index = False)

In [20]:
main()

loaded data:
  Tache Description    P  Duree  Marge Total  Critique  Unnamed: 6
0     A  Activity 1  NaN      6          NaN       NaN         NaN
1     B  Activity 2  NaN      9          NaN       NaN         NaN
2     C  Activity 3  A,B     10          NaN       NaN         NaN
3     D  Activity 4    C      8          NaN       NaN         NaN
4     E  Activity 5    C      5          NaN       NaN         NaN
5     F  Activity 6  E,D      7          NaN       NaN         NaN
6     G  Activity 7    F      5          NaN       NaN         NaN
7     H  Activity 8    F      8          NaN       NaN         NaN
results:
  Tache Description    P  Duree  Marge Totale Critique
0     A  Activity 1  NaN      6             3      Non
1     B  Activity 2  NaN      9             0      Oui
2     C  Activity 3  A,B     10             0      Oui
3     D  Activity 4    C      8             0      Oui
4     E  Activity 5    C      5             3      Non
5     F  Activity 6  E,D      7             0